<a href="https://colab.research.google.com/github/francescogrillea/AnomalyDetectionTFLite/blob/main/AnomalyDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import e Costanti

In [ ]:
!rm ./drive/MyDrive/Colab*/anomaly_detection_data/TFLite_Models/LSTM_1L/*

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
import time

PROJECT_NAME = 'anomaly_detection'
PROJECT_PATH = './drive/MyDrive/Colab Notebooks/anomaly_detection_data/'
DATASET_PATH = PROJECT_PATH+'Dataset/'
MODEL_TYPE = 'LSTM_1L/'
TF_MODELS_PATH = PROJECT_PATH+'TF_Models/'+MODEL_TYPE
TFLite_MODELS_PATH = PROJECT_PATH+'TFLite_Models/'+MODEL_TYPE
CHANNEL_NAME = 'A-1'

# Load Model and Dataset

In [ ]:
def load_tf_model(channel_name):
  if not channel_name.endswith('.h5'):
    channel_name = channel_name+'.h5'
  tf_model = keras.models.load_model(TF_MODELS_PATH+channel_name)
  return tf_model


def load_dataset(channel_name):
  if not channel_name.endswith('.npy'):
    channel_name = channel_name+'.npy'
  x = np.array([np.load(DATASET_PATH+'test/'+channel_name)])
  y = np.array([])
  return x,y

# Prediction su TF Model

In [ ]:
#load dataset
x_tf, tf_output_data = load_dataset(CHANNEL_NAME)

model = load_tf_model(CHANNEL_NAME)
tf_output_data = model.predict(x_tf)
print(tf_output_data)

[[0.9981911  0.9989598  0.99894804 0.99897206 0.9992333  0.99905074
  0.9991313  0.99830365 0.9991017  0.99896634]]


# Conversione dei modelli da .h5 a .tflite

In [ ]:
channels = [f for f in os.listdir(TF_MODELS_PATH)]

for model_filename in channels:
  if not model_filename.startswith(CHANNEL_NAME):
    continue

  tf_model = load_tf_model(model_filename)
  converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
  converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
  #optimizations = [tf.lite.Optimize.DEFAULT]
  tflite_model = converter.convert()

  #generate TFLite model name
  channel_name = model_filename[:-3]
  tflite_model_filename = channel_name+'.tflite'
  
  TFLite_MODEL_FILE = TFLite_MODELS_PATH+tflite_model_filename
  with open(TFLite_MODEL_FILE, 'wb') as f:
    f.write(tflite_model)
    print('Model for channel '+channel_name+' converted')
    #break

INFO:tensorflow:Assets written to: /tmp/tmph5005zwy/assets


INFO:tensorflow:Assets written to: /tmp/tmph5005zwy/assets


Model for channel A-1 converted


# Inferenza su TFLite

In [ ]:
#load dataset
x_data, tflite_output_data = load_dataset(CHANNEL_NAME)

interpreter = tf.lite.Interpreter(model_path=TFLite_MODELS_PATH+CHANNEL_NAME+'.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#preprocessing input data
input_shape = input_details[0]['shape']

#per ogni riga del dataset, creo un tensore e invoco il modello
for data in x_data[0]:
  input_data = np.array([[data]], dtype=np.float32)
  interpreter.set_tensor(input_details[0]['index'], input_data)
    
  #run inference
  start_time = time.time()
  interpreter.invoke()
  stop_time = time.time()
  
  #recupero l'output prodotto dal modello
  output_data = interpreter.get_tensor(output_details[0]['index'])
  tflite_output_data = np.append(tflite_output_data, output_data)

print(tflite_output_data)

[0.99269593 0.99599427 0.99681348 ... 0.99395204 0.99885583 0.99699628]


In [ ]:
#confronto tra i risultati prodotti dal modello TF (sinistra) rispetto al modello TFLite (destra)
for y_tf, y_tflite in zip(tf_output_data[0], tflite_output_data):
  print('{} -- {}: error {}'.format(y_tf, y_tflite, y_tf-y_tflite))

0.9981911182403564 -- 0.9926959276199341: error 0.005495190620422363
0.9989597797393799 -- 0.9959942698478699: error 0.0029655098915100098
0.9989480376243591 -- 0.9968134760856628: error 0.002134561538696289
0.9989720582962036 -- 0.9970735907554626: error 0.0018984675407409668
0.9992333054542542 -- 0.9978994131088257: error 0.0013338923454284668
0.9990507364273071 -- 0.9971656203269958: error 0.0018851161003112793
0.9991313219070435 -- 0.9977410435676575: error 0.0013902783393859863
0.9983036518096924 -- 0.9939520359039307: error 0.004351615905761719
0.9991016983985901 -- 0.9988558292388916: error 0.00024586915969848633
0.9989663362503052 -- 0.996996283531189: error 0.001970052719116211
